## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_30_copy.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_30_copy.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "배란_유도_유형",
    "단일_배아_이식_여부",
    "착상_전_유전_진단_사용_여부",
    "배아_생성_주요_이유",
    "총_생성_배아_수",
    "미세주입된_난자_수",
    "미세주입에서_생성된_배아_수",
    "이식된_배아_수",
    "미세주입_배아_이식_수",
    "저장된_배아_수",
    "미세주입_후_저장된_배아_수",
    "해동된_배아_수",
    "해동_난자_수",
    "수집된_신선_난자_수",
    "저장된_신선_난자_수",
    "혼합된_난자_수",
    "파트너_정자와_혼합된_난자_수",
    "기증자_정자와_혼합된_난자_수",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "동결_배아_사용_여부",
    "신선_배아_사용_여부",
    "기증_배아_사용_여부",
    "대리모_여부",
    "ICSI_배아_이식_비율",
    "ICSI_배아_생성_비율",
    "ICSI_성공률"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [7]:
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),  # 기본값: 100
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True),  # 기본값: 0.3
        'max_depth': trial.suggest_int('max_depth', 1, 15),  # 기본값: 6
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # 기본값: 1
        'gamma': trial.suggest_float('gamma', 0, 5),  # 기본값: 0
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # 기본값: 1
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # 기본값: 1
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 10.0, log=True),  # 기본값: 0
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 10.0, log=True),  # 기본값: 1
        
        'random_state': 42,
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'use_label_encoder': False,
        'verbosity': 0,
        'n_jobs': -1
    }

    model = xgb.XGBClassifier(**param)
    model.fit(Total_X_train, Total_y_train, eval_set=[(Total_X_test, Total_y_test)], verbose=False)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1200)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-15 08:28:24,484] A new study created in memory with name: no-name-244733b4-5e41-4d81-b580-57959454929b
[I 2025-02-15 08:35:27,964] Trial 0 finished with value: 0.7332533262315769 and parameters: {'n_estimators': 2213, 'learning_rate': 2.7045131701954626e-05, 'max_depth': 9, 'min_child_weight': 5, 'gamma': 0.5603859092912555, 'subsample': 0.510710140910714, 'colsample_bytree': 0.7993423363993934, 'reg_alpha': 8.03788719493011e-05, 'reg_lambda': 5.000500414927698}. Best is trial 0 with value: 0.7332533262315769.
[I 2025-02-15 09:08:18,619] Trial 1 finished with value: 0.7364722178213352 and parameters: {'n_estimators': 6958, 'learning_rate': 0.0003514010575028525, 'max_depth': 12, 'min_child_weight': 8, 'gamma': 4.399564868126501, 'subsample': 0.6383901706326646, 'colsample_bytree': 0.8107178272405731, 'reg_alpha': 0.0024173787508648577, 'reg_lambda': 0.0011660718718697477}. Best is trial 1 with value: 0.7364722178213352.
[I 2025-02-15 09:34:03,291] Trial 2 finished with value

KeyboardInterrupt: 

.